In [ ]:
# for pretty code
%load_ext nb_black

# Imports and functions

In [1]:
# imports
import numpy as np
import pandas as pd
import statsmodels.api as sm

from newsrelations.query_db.relation_query import DBQueryHandlerCoocc
from newsrelations.helper_classes.synonym_handler import SynonymHandler
from newsrelations.metrics.distances import DistanceMeasure

In [2]:
def build_contingency_table_from_single_topic(
    relation_models_path, relation_models, topic_of_interest, no_entities=10
):
    """
    This function builds a contingency table from a input list of relation models generated with relation_miner.py 
    in regards to predetermined topic [TOPIC_OF_INTEREST].
    The first model in [RELATION_MODELS] is the reference model all other models will be compared with.
    The function extracts the top [NO_ENTITIES] co_occuring entities from the model and builds a contingency table.
    
    input:  relation_models = list
            relation_models_path = str 
            topic_of_interest = str           
            no_entities = int (standard 10)
            
    output: contingency_table = pandas DataFrame [rows = different models, columns = entities]
    """
    # initialize DistanceMeasure with reference-model
    dm = DistanceMeasure(relation_models_path, str(relation_models[0]))

    # extract top NO_ENTITIES entities
    top = dm.get_top_co_occurrences(
        topic_of_interest, cutoff=no_entities, e1_is_synset=0, e2_is_synset=0
    )
    # write first row of contingency_table
    contingency_table = pd.DataFrame(
        np.array([t[1] for t in top]),
        index=[t[0] for t in top],
        columns=[str(relation_models[0])],
    )

    # loop through all remaining models
    for model in relation_models[1:]:
        # initialize db_handler()
        db_handler = DBQueryHandlerCoocc(relation_models_path, model)

        # buffer for cooccurrences
        co_occs = []
        # loop through all entities and get number of co-occurrences
        for row in contingency_table.index:
            co_occs.append(
                len(
                    list(
                        db_handler.select_relations(
                            e1=topic_of_interest.lower(),
                            e2=row.lower(),
                            e1_is_synset=0,
                            e2_is_synset=0,
                        )
                    )
                )
            )

        contingency_table[str(model)] = co_occs

    # transpose the contingency table to get it into the right format
    contingency_table = contingency_table.transpose()
    return contingency_table

In [4]:
def explain_models(relation_models):
    """
    This function lists the models from the relation_models list and their counter index
    
    Input: relation_models = list
    
    Output: None
    """
    i = 0
    for model in relation_models:
        print("model (" + str(i) + "): " + str(model))
        i += 1
    return

In [3]:
def build_contingency_table_from_topic_list(
    relation_models_path, relation_models, topic_of_interest_list, no_entities=10
):
    """
    This function builds a contingency table from a input list of relation models generated with relation_miner.py 
    in regards to predetermined topic list [topic_of_interest_list].
    The first model in [relation_models] is the reference model all other models will be compared with.
    The function extracts the top [no_entities] co-occuring entities for the first [topic_of_interest] from the 
    model and builds a contingency table.
    
    
    input:  relation_models = list
            relation_models_path = str 
            topic_of_interest_list = list           
            no_entities = int (standard 10)
            
    output: contingency_table = pandas DataFrame [rows = different models, columns = entities]
    """
    # identifier for models in relation_models_list
    i = 0
    # print models with idx 
    explain_models(relation_models)
    
    # initialize DistanceMeasure with reference-model
    dm = DistanceMeasure(relation_models_path, str(relation_models[0]))

    # extract top NO_ENTITIES entities
    top = dm.get_top_co_occurrences(
        topic_of_interest_list[0], cutoff=no_entities, e1_is_synset=0, e2_is_synset=0
    )
    # write first row of contingency_table
    contingency_table = pd.DataFrame(
        np.array([t[1] for t in top]),
        index=[t[0] for t in top],
        columns=[str(topic_of_interest_list[0]) + " (" + str(i) + ")"],
    )

    # loop through the models
    for model in relation_models[:]:
        # initialize db_handler()
        db_handler = DBQueryHandlerCoocc(relation_models_path, model)

        for topic in topic_of_interest_list:
            # buffer for co-occurrencces
            co_occs = []

            # loop through all all entities and get number of co-occurrences
            for row in contingency_table.index:
                co_occs.append(
                    len(
                        list(
                            db_handler.select_relations(
                                e1=topic.lower(),
                                e2=row.lower(),
                                e1_is_synset=0,
                                e2_is_synset=0,
                            )
                        )
                    )
                )
            contingency_table[str(topic) + " (" + str(i) + ")"] = co_occs

        i += 1

    # transpose the contingency table to get it into the right format
    contingency_table = contingency_table.transpose()

    return contingency_table

In [5]:
def compare_entity_lists(
    relation_models_path, relation_models, topic_list1, topic_list2
):
    """
    This function creates a contingency table from a two input lists of entities [topic_list1][topic_list2]
    from a list of relation models 
    
    input:  relation_models = list
            relation_models_path = str 
            topic_list1 = list           
            topic_list2 = list
            
    output: contingency_table = pandas DataFrame [rows = different models, columns = entities]
    """
    # idx for models
    i = 0
    
    # print models with idx 
    explain_models(relation_models)

    # initialize contingency_table
    contingency_table = pd.DataFrame(index=topic_list2)

    # loop through the models
    for model in relation_models:
        # initialize db_handler()
        db_handler = DBQueryHandlerCoocc(relation_models_path, model)

        for topic1 in topic_list1:
            # buffer for co-occurrencces
            co_occs = []

            # loop through all all entities and get number of co-occurrences
            for topic2 in topic_list2:
                co_occs.append(
                    len(
                        list(
                            db_handler.select_relations(
                                e1=topic1.lower(),
                                e2=topic2.lower(),
                                e1_is_synset=0,
                                e2_is_synset=0,
                            )
                        )
                    )
                )
            contingency_table[str(topic1) + " (" + str(i) + ")"] = co_occs

        i += 1

    # transpose the contingency table to get it into the right format
    contingency_table = contingency_table.transpose()

    return contingency_table

In [6]:
def chi_squared(contingency_table):
    """
    This function conducts a chi-squared test of independence between the different rows of a contingency table
    
    input: contingency_table
    
    ouput: None
    """
    contingency_table = sm.stats.Table(contingency_table)
    results = contingency_table.test_nominal_association()
    
    
    # orig contingency table
    print("Original contingency table:")
    print(contingency_table.table_orig)
    # expected values
    print("\nExpected values:")
    print(contingency_table.fittedvalues)
    # residual 
    print("\nChi-square contributions:")
    print(contingency_table.chi2_contribs)
    print("\nResults:")
    
    print(results)
    
    
    results = contingency_table.test_ordinal_association()
    #print("\nOrdinal test results:")
    #print(results)
    
    return


# Variables

Variable setups for different runs of models (all scraped from commoncrawl.com)

In [ ]:
# First Try 
# general tryout on the NewsRelations Library

# directory path of relation models 
RELATION_MODELS_PATH = "/home/jonas/Documents/GitHub/MasterThesis/models/firstTry"

# model name
RELATION_MODELS = [
    "model.sqlite",
]

In [ ]:
# Second Try
# models from newssources with different biases 
#
# -year:    2008 (for foxnews 2o08-2010)
# -domain:  politics
# -sources: NYT & foxnews 

# directory path of relation models 
RELATION_MODELS_PATH = "/home/jonas/Documents/GitHub/MasterThesis/models/secondTry"

# model names
RELATION_MODELS = [
    "RMadvanced_2008_politics_nytimes.sqlite",
    "RM_2009_politics_nytimes.sqlite",
    "RM_2008-2010_politics_foxnews.sqlite",
]



In [7]:
# Third Try
# models from newssources with different biases 
#
# -year:    2011
# -domain:  news
# -sources: reuters & national public radio

# directory path of relation models 
RELATION_MODELS_PATH = "/home/jonas/Documents/GitHub/MasterThesis/models/thirdTry"

# model names
RELATION_MODELS = [
    "RM_2011_news_reuters.sqlite",
    "RM_2011_news_npr.sqlite",
]



In [ ]:
# Fourth try
# two models from newssources with different biases each, for comparing slant coherence within
# different directions. Timeslots with gapless news reporting were chosen.
#
# -year:    2011-01-01 - 2011-03-31
# -domain:  news
# -sources(left):    New York Times (NYT) 637, Washington Post (WP) 508
#         (center):  National Public Radio (NPR) 109, Reuters (RET) 300
#         (right):   FoxNews (FN) 2735, Newsmax (NM) 180

# directory path of relation models
RELATION_MODELS_PATH = "/home/jonas/Documents/GitHub/MasterThesis/models/fourthTry"

# model names
RELATION_MODELS = [
    "RM_2011_news_NYT.sqlite",
    "RM_2011_news_WP.sqlite",
    "RM_2011_news_RET.sqlite",
    "RM_2011_news_NPR.sqlite",
    "RM_2011_news_FN.sqlite",
    "RM_2011_news_NM.sqlite",
]

general global variables 

In [9]:
# number of entities you want to compare
NO_ENTITIES = 5

# reference entity
TOPIC_OF_INTEREST = "united_states"

# reference entity list
TOPIC_OF_INTEREST_LIST = [
    "united_states", 
    "germany", 
    "russia", 
    "india", 
    "china"
]

# co-occurrence entity list
TOPIC_OF_INTEREST_LIST2 = [
    "united_states",
    "united_states",
    "united_states",
    "united_states",
    "united_states",
]

# Tests

In [10]:
# extracting a contingency table from a single reference entity 
contingency_table = build_contingency_table_from_single_topic(
    RELATION_MODELS_PATH, RELATION_MODELS, TOPIC_OF_INTEREST, NO_ENTITIES
)

chi_squared(contingency_table)

Original contingency table:
                             reuters  u.s.  washington  new_york  china
RM_2011_news_reuters.sqlite     1725  1286         724       478    399
RM_2011_news_npr.sqlite            3   388         247       126     88

Expected values:
                                 reuters         u.s.  washington    new_york  \
RM_2011_news_reuters.sqlite  1458.553441  1412.973646   819.59224  509.818448   
RM_2011_news_npr.sqlite       269.446559   261.026354   151.40776   94.181552   

                                  china  
RM_2011_news_reuters.sqlite  411.062225  
RM_2011_news_npr.sqlite       75.937775  

Chi-square contributions:
                                reuters       u.s.  washington   new_york  \
RM_2011_news_reuters.sqlite   48.674095  11.410196   11.149296   1.985832   
RM_2011_news_npr.sqlite      263.479961  61.765053   60.352761  10.749596   

                                china  
RM_2011_news_reuters.sqlite  0.353954  
RM_2011_news_npr.sqlite      

In [11]:
# extract a contingency table from a list of reference entities 
contingency_table = build_contingency_table_from_topic_list(
    RELATION_MODELS_PATH, RELATION_MODELS, TOPIC_OF_INTEREST_LIST, NO_ENTITIES)

chi_squared(contingency_table)

Original contingency table:
                   reuters  u.s.  washington  new_york  china
united_states (0)     1725  1286         724       478    399
germany (0)            327   175          56        84    112
russia (0)             274   140          65        42    136
india (0)              282   178          64        73    181
china (0)              812   484         201       210    518
united_states (1)        3   388         247       126     88
germany (1)              2    55          32        26     19
russia (1)               2    48          28        17     26
india (1)                0    58          33        26     55
china (1)                1   123          63        35    105

Expected values:
                       reuters         u.s.  washington    new_york  \
united_states (0)  1487.161251  1273.098519  656.285540  484.514837   
germany (0)         243.130872   208.134493  107.293863   79.211662   
russia (0)          211.852763   181.358570   93.490806   6

In [ ]:
# extract a contingency table from two lists of entities 

contingency_table = compare_entity_lists(
    RELATION_MODELS_PATH,
    RELATION_MODELS,
    TOPIC_OF_INTEREST_LIST,
    TOPIC_OF_INTEREST_LIST2,
)

chi_squared(contingency_table)

model (0): RM_2011_news_reuters.sqlite
model (1): RM_2011_news_npr.sqlite


In [ ]:
    # initialize DistanceMeasure with reference-model
dm = DistanceMeasure(RELATION_MODELS_PATH, str(RELATION_MODELS[0]))

    # extract top NO_ENTITIES entities
topNYT = dm.get_top_co_occurrences(
    TOPIC_OF_INTEREST, cutoff=10, e1_is_synset=0, e2_is_synset=0
)

dm = DistanceMeasure(RELATION_MODELS_PATH, str(RELATION_MODELS[1]))

    # extract top NO_ENTITIES entities
topFN = dm.get_top_co_occurrences(
    TOPIC_OF_INTEREST, cutoff=10, e1_is_synset=0, e2_is_synset=0
)


print(topNYT)
print(topFN)
